<a href="https://colab.research.google.com/github/PiyushGithub24/MACHINE_LEARNING_BY_PIYUSH_KUMAR/blob/main/Day_18(b)Fetching_Data_From_Web_Scraping(30_05_24).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fetching Data From Ambition box using Web Scraping :--

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup     #bs4:-beautifulsoup4

In [2]:
requests.get("https://www.ambitionbox.com/list-of-companies?page=1")

<Response [403]>

In [3]:
requests.get("https://www.ambitionbox.com/list-of-companies?page=1").text

'<HTML><HEAD>\n<TITLE>Access Denied</TITLE>\n</HEAD><BODY>\n<H1>Access Denied</H1>\n \nYou don\'t have permission to access "http&#58;&#47;&#47;www&#46;ambitionbox&#46;com&#47;list&#45;of&#45;companies&#63;" on this server.<P>\nReference&#32;&#35;18&#46;6424c317&#46;1717597898&#46;2e030e87\n<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;6424c317&#46;1717597898&#46;2e030e87</P>\n</BODY>\n</HTML>\n'

In [4]:
url1="https://www.ambitionbox.com/list-of-companies?page=1"

In [5]:
def create_df(url):
  headers=headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
  webpage=requests.get(url,headers=headers).text
  soup=BeautifulSoup(webpage,'lxml')
  company=soup.find_all('div',class_='companyCardWrapper')

  names=[]
  reviews=[]
  rating=[]
  industry=[]
  employees=[]
  ctype=[]
  old=[]
  hq=[]

  for i in company:
    names.append(i.find('h2').text.strip())
    rating.append(i.find('span',class_="companyCardWrapper__companyRatingValue").text)
    reviews.append(i.find('span',class_="companyCardWrapper__ActionCount").text)
    info=(i.find('span', class_="companyCardWrapper__interLinking")).text.strip()
    info=info.split('|')
    if len(info)==5:
      industry.append(info[0])
      employees.append(info[1])
      ctype.append(info[2])
      old.append(info[3])
      hq.append(info[4])
    else:
      for i in info:
        if 'Employees' in i:
          employees.append(i)
          break
      else:
          employees.append(None)

      for i in info:
        if 'old' in i:
          old.append(i)
          break
      else:
          old.append(None)

      for i in info:
        if "more" in i:
          hq.append(i)
          break
      else:
          hq.append(None)

      industry.append(info[0])

      c=['Public','Startup','Central','Private ','Partnership','Cooperative']
      found=False
      for element in c:
        for sentence in info:
            if element in sentence:
              ctype.append(sentence)
              found=True
              break
        if found:
          break
      else:
        ctype.append(None)
  comp_dict={'Company_names':names,"rating":rating,"Reviews":reviews,"Industry":industry,"Employess":employees,"Company_type":ctype,"Company_Old_Years":old,"Headquarter":hq}
  comp_df=pd.DataFrame(comp_dict)
  return comp_df

In [9]:
company_dataframe=pd.DataFrame()
for i in range(1,101):
  url="https://www.ambitionbox.com/list-of-companies?page={}".format(i)
  company_dataframe=pd.concat([company_dataframe,create_df(url)],ignore_index=True)


In [10]:
company_dataframe.shape

(2000, 8)

In [11]:
company_dataframe.head()

,Company_names,rating,Reviews,Industry,Employess,Company_type,Company_Old_Years,Headquarter
0,TCS,3.8,75.1k,IT Services & Consulting,1 Lakh+ Employees,Public,56 years old,Mumbai +346 more
1,Accenture,4.0,46.8k,IT Services & Consulting,1 Lakh+ Employees,Public,35 years old,Dublin +180 more
2,Cognizant,3.9,42k,IT Services & Consulting,1 Lakh+ Employees,Forbes Global 2000,30 years old,Teaneck. New Jersey. +169 more
3,Wipro,3.8,40.4k,IT Services & Consulting,1 Lakh+ Employees,Public,79 years old,Bangalore/Bengaluru +274 more
4,Capgemini,3.8,34.7k,IT Services & Consulting,1 Lakh+ Employees,Public,57 years old,Paris +139 more


In [12]:
company_dataframe.tail()

,Company_names,rating,Reviews,Industry,Employess,Company_type,Company_Old_Years,Headquarter
1995,Marble Box,4.2,378,BPO,501-1k Employees,MNC,22 years old,Kolkata +5 more
1996,Blackrock,3.9,378,Management Consulting,1k-5k Employees,Forbes Global 2000,20 years old,New York City +11 more
1997,People Tech Group,2.8,376,IT Services & Consulting,1k-5k Employees,None,18 years old,Redmond +9 more
1998,V.S.T Tillers Tractors,3.6,357,Industrial Machinery,201-500 Employees,Public,57 years old,Bangalore/Bengaluru +45 more
1999,Iron Mountain Inc,3.8,350,IT Services & Consulting,1k-5k Employees,None,73 years old,Boston +36 more


In [13]:
company_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Company_names      2000 non-null   object
 1   rating             2000 non-null   object
 2   Reviews            2000 non-null   object
 3   Industry           2000 non-null   object
 4   Employess          1980 non-null   object
 5   Company_type       1006 non-null   object
 6   Company_Old_Years  1931 non-null   object
 7   Headquarter        2000 non-null   object
dtypes: object(8)
memory usage: 125.1+ KB


#Now I successfully fetch all required data of companies from many pages!!!

In [14]:
#Let's create csv file of the dataframe:--
company_dataframe.to_csv('Companies_Related_Data.csv')